# **Практика: полиномиальная регрессия и регуляризация**

✍ Продолжим решать задачу от владельца компании «Газ-Таз-Ваз-Нефть» Василия. Исходный датасет представлен [здесь](https://lms.skillfactory.ru/assets/courseware/v1/11a2ef69ecca8fc5ec1b5c43c8dde935/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/unconv.zip).

Ранее, в юните 5, мы смогли построить модель линейной регрессии, которая прогнозирует выработку газа на скважине. Для этого мы с помощью матрицы корреляций и рассуждений отобрали некоррелированные, значимые для предсказания признаки. Далее мы будем использовать именно их (см. задание 5.5).

Мы хотим попробовать улучшить наш результат — метрику **MAPE**. Для этого воспользуемся моделью **полиномиальной регрессии третьей степени**. Однако теперь мы знаем, что полиномиальным моделям очень легко переобучиться под исходную выборку, поэтому для контроля качества модели мы будем использовать **кросс-валидацию**.

Приступим!

In [20]:
import numpy as np 
import pandas as pd 
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

data = pd.read_csv('data/unconv.csv')
data.head()

X = data.drop(['Prod','Well','Perm','TOC'], axis=1)
y = data[['Prod']]

Сгенерируйте полиномиальные признаки третьего порядка на факторах, которые вы выбрали для обучения моделей. Для этого воспользуйтесь генератором полиномов PolynomialFeatures из библиотеки sklearn. Параметр include_bias установите в значение False.

1. Сколько факторов у вас получилось после генерации полиномиальных признаков?

In [22]:
poly = PolynomialFeatures(degree=3, include_bias=False)
X_poly = poly.fit_transform(X)
display(pd.DataFrame(X_poly))

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,12.08,2.80,81.40,2.31,145.9264,33.8240,983.3120,27.9048,7.8400,227.9200,...,21.952000,638.176000,18.110400,18552.688000,526.495200,14.941080,539353.144000,15305.967600,434.358540,12.326391
1,12.38,3.22,46.17,1.88,153.2644,39.8636,571.5846,23.2744,10.3684,148.6674,...,33.386248,478.709028,19.492592,6863.973858,279.494712,11.380768,98419.153113,4007.537532,163.183248,6.644672
2,14.02,4.01,72.80,2.72,196.5604,56.2202,1020.6560,38.1344,16.0801,291.9280,...,64.481201,1170.631280,43.737872,21252.358400,794.044160,29.667584,385828.352000,14415.564800,538.603520,20.123648
3,17.67,2.63,39.81,1.88,312.2289,46.4721,703.4427,33.2196,6.9169,104.7003,...,18.191447,275.361789,13.003772,4168.118943,196.836564,9.295472,63092.325141,2979.491868,140.704464,6.644672
4,17.52,3.18,10.94,1.90,306.9504,55.7136,191.6688,33.2880,10.1124,34.7892,...,32.157432,110.629656,19.213560,380.593848,66.099480,11.479800,1309.338584,227.398840,39.493400,6.859000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,11.95,2.97,67.18,2.06,142.8025,35.4915,802.8010,24.6170,8.8209,199.5246,...,26.198073,592.588062,18.171054,13404.062628,411.020676,12.603492,303193.578232,9297.093944,285.085048,8.741816
196,17.99,3.38,44.32,2.08,323.6401,60.8062,797.3168,37.4192,11.4244,149.8016,...,38.614472,506.329408,23.762752,6639.206912,311.587328,14.623232,87056.109568,4085.665792,191.746048,8.998912
197,12.12,3.52,57.07,1.73,146.8944,42.6624,691.6884,20.9676,12.3904,200.8864,...,43.614208,707.120128,21.435392,11464.586848,347.533472,10.535008,185876.128243,5634.583877,170.804803,5.177717
198,15.55,2.48,58.25,2.35,241.8025,38.5640,905.7875,36.5425,6.1504,144.4600,...,15.252992,358.260800,14.453440,8414.795000,339.481000,13.695800,197645.890625,7973.696875,321.685625,12.977875


Обучите модель линейной регрессии из библиотеки sklearn (LinearRegression) на полученных полиномиальных факторах.

Оцените среднее значение метрики MAPE, используя кросс-валидацию на пяти фолдах.

2. Чему равны средние значения метрики MAPE на тренировочных и валидационных фолдах? Ответ приведите в процентах (не указывайте знак процента), округлив его до первого знака после точки-разделителя.

In [26]:
# создаём модель линейной регрессии
LinR = LinearRegression()
# оцениваем качество модели на кросс-валидации
cv_results = cross_validate(LinR, X_poly, y, scoring='neg_mean_absolute_percentage_error', cv=5, return_train_score=True)
print('MAPE на тренировочных фолдах: {:.1f} %'.format(-cv_results['train_score'].mean()* 100))
print('MAPE на валидационных фолдах: {:.1f} %'.format(-cv_results['test_score'].mean() * 100)) 

MAPE на тренировочных фолдах: 1.8 %
MAPE на валидационных фолдах: 2.7 %


Теперь попробуем воспользоваться линейной регрессией с регуляризацией. Для начала возьмём L1-регуляризацию.

Обучите модель Lasso из библиотеки sklearn на полученных полиномиальных факторах, предварительно стандартизировав факторы с помощью StandardScaler. Коэффициент регуляризации выставите равным 5.

Оцените среднее значение метрики MAPE, используя кросс-валидацию на пяти фолдах.

Чему равны средние значения метрики MAPE на тренировочных и валидационных фолдах? Ответ приведите в процентах (не указывайте знак процента), округлив его до первого знака после точки-разделителя.

In [30]:
X = data.drop(['Prod','Well','Perm','TOC'], axis=1)
y = data[['Prod']]
# инициализируем стандартизатор StandardScaler
scaler = StandardScaler()
# подгоняем параметры стандартизатора (вычисляем среднее и СКО)
X = scaler.fit_transform(X)

# добавляем полиномиальные признаки
poly = PolynomialFeatures(degree=3, include_bias=False)
X = poly.fit_transform(X)

# создаём модель линейной регрессии c L1-регуляризацией
lasso = Lasso(alpha=5)

# оцениваем качество модели на кросс-валидации
cv_results = cross_validate(lasso, X, y, scoring='neg_mean_absolute_percentage_error', cv=5, return_train_score=True)
print('MAPE на тренировочных фолдах: {:.1f} %'.format(-cv_results['train_score'].mean()* 100))
print('MAPE на валидационных фолдах: {:.1f} %'.format(-cv_results['test_score'].mean() * 100))

MAPE на тренировочных фолдах: 1.8 %
MAPE на валидационных фолдах: 2.3 %


Проделаем то же самое с L2-регуляризацией.

Обучите модель Ridge из библиотеки sklearn на полученных полиномиальных факторах, предварительно стандартизировав факторы с помощью StandardScaler. Коэффициент регуляризации выставите равным 1.

Оцените среднее значение метрики MAPE, используя кросс-валидацию на пяти фолдах.

Чему равны средние значения метрики MAPE на тренировочных и валидационных фолдах? Ответ приведите в процентах (не указывайте знак процента), округлив его до первого знака после точки-разделителя.

In [31]:
X = data.drop(['Prod','Well','Perm','TOC'], axis=1)
y = data[['Prod']]
# инициализируем стандартизатор StandardScaler
scaler = StandardScaler()
# подгоняем параметры стандартизатора (вычисляем среднее и СКО)
X = scaler.fit_transform(X)

# добавляем полиномиальные признаки
poly = PolynomialFeatures(degree=3, include_bias=False)
X = poly.fit_transform(X)

# создаём модель линейной регрессии c L2-регуляризацией
ridge = Ridge(alpha=1)

# оцениваем качество модели на кросс-валидации
cv_results = cross_validate(ridge, X, y, scoring='neg_mean_absolute_percentage_error', cv=5, return_train_score=True)
print('MAPE на тренировочных фолдах: {:.1f} %'.format(-cv_results['train_score'].mean()* 100))
print('MAPE на валидационных фолдах: {:.1f} %'.format(-cv_results['test_score'].mean() * 100))

MAPE на тренировочных фолдах: 1.8 %
MAPE на валидационных фолдах: 2.7 %
